# Subgraph (직접 연결): 그래프 안의 그래프

이 노트북에서는 **Subgraph**를 사용하여 **그래프 안에 그래프를 넣는** 방법을 배웁니다.

## Subgraph란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Subgraph의 개념                                  │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   서브그래프 = 부모 그래프 안에 포함된 자식 그래프                 │
│                                                                    │
│   ┌─────────────── 부모 그래프 ───────────────┐                   │
│   │                                          │                    │
│   │   START → ┌─────────────────┐ → END      │                    │
│   │           │   서브그래프     │            │                    │
│   │           │ ┌───┐   ┌───┐  │            │                    │
│   │           │ │ A │ → │ B │  │            │                    │
│   │           │ └───┘   └───┘  │            │                    │
│   │           └─────────────────┘            │                    │
│   │                                          │                    │
│   └──────────────────────────────────────────┘                    │
│                                                                    │
│   장점:                                                            │
│   • 복잡한 로직을 모듈화                                           │
│   • 재사용 가능한 그래프 컴포넌트                                  │
│   • 코드 관리 용이                                                 │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 이번 노트북: 직접 연결 방식

```
┌────────────────────────────────────────────────────────────────────┐
│                    직접 연결 vs 함수 래핑                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   [직접 연결] (이번 노트북)                                        │
│   • 부모와 자식이 상태 키를 공유                                   │
│   • builder.add_node('name', subgraph)                            │
│   • 간단하지만 유연성 낮음                                         │
│                                                                    │
│   [함수 래핑] (다음 노트북)                                        │
│   • 상태 키가 달라도 됨                                            │
│   • 함수 안에서 상태 변환                                          │
│   • 더 유연함                                                      │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langgraph

# 2. State 정의

부모 그래프와 서브그래프의 상태를 정의합니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    상태 공유 구조                                   │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   부모 State:                     서브그래프 State:                │
│   ┌─────────────┐                ┌─────────────┐                  │
│   │ foo: str    │ ◄──── 공유 ────► foo: str    │                  │
│   └─────────────┘                │ bar: str    │ ← 추가 필드       │
│                                  └─────────────┘                  │
│                                                                    │
│   'foo' 키를 통해 부모 ↔ 서브그래프 통신                          │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from typing import TypedDict

# 부모 그래프의 상태
class State(TypedDict):
    foo: str  # 서브그래프와 공유되는 키

# 서브그래프의 상태
class SubgraphState(TypedDict):
    foo: str  # 부모 그래프와 공유되는 키
    bar: str  # 서브그래프 내부에서만 사용

print("✅ State 정의 완료")
print("   - State (부모): foo")
print("   - SubgraphState: foo (공유), bar (내부용)")

# 3. 서브그래프 정의

In [ ]:
from langgraph.graph import StateGraph, START

def subgraph_node(state: SubgraphState):
    """
    서브그래프의 노드
    
    공유 키인 'foo'를 수정하여 부모 그래프와 통신
    """
    print(f"  [서브그래프] 입력 foo: {state['foo']}")
    
    # foo 값에 'bar' 추가
    new_foo = state['foo'] + 'bar'
    
    print(f"  [서브그래프] 출력 foo: {new_foo}")
    return {'foo': new_foo}


# 서브그래프 빌더
subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node('subgraph_node', subgraph_node)
subgraph_builder.add_edge(START, 'subgraph_node')

# 서브그래프 컴파일
subgraph = subgraph_builder.compile()

print("✅ 서브그래프 정의 완료")

In [ ]:
# 서브그래프 단독 테스트
print("=== 서브그래프 단독 테스트 ===")
result = subgraph.invoke({'foo': 'hello', 'bar': ''})
print(f"결과: {result}")

# 4. 부모 그래프 정의 (서브그래프 직접 연결)

In [ ]:
# 부모 그래프 빌더
builder = StateGraph(State)

# 서브그래프를 노드로 직접 추가!
builder.add_node('subgraph', subgraph)  # 컴파일된 서브그래프를 노드로 추가
builder.add_edge(START, 'subgraph')

# 부모 그래프 컴파일
graph = builder.compile()

print("✅ 부모 그래프 정의 완료")
print("   서브그래프가 'subgraph' 노드로 추가됨")

In [ ]:
# 그래프 구조 시각화
print("=== 부모 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 5. 그래프 실행

In [ ]:
# 부모 그래프 실행
initial_state = {'foo': 'hello'}

print("=== 부모 그래프 실행 ===")
print(f"입력: {initial_state}")
print()

result = graph.invoke(initial_state)

print()
print(f"출력: {result}")
print()
print("→ 'hello' + 'bar' = 'hellobar'")

# 6. 여러 서브그래프 연결

In [ ]:
# 두 번째 서브그래프 정의
def subgraph_node2(state: SubgraphState):
    print(f"  [서브그래프2] 입력 foo: {state['foo']}")
    new_foo = state['foo'] + '!'
    print(f"  [서브그래프2] 출력 foo: {new_foo}")
    return {'foo': new_foo}

subgraph_builder2 = StateGraph(SubgraphState)
subgraph_builder2.add_node('subgraph_node2', subgraph_node2)
subgraph_builder2.add_edge(START, 'subgraph_node2')
subgraph2 = subgraph_builder2.compile()

print("✅ 두 번째 서브그래프 정의 완료")

In [ ]:
from langgraph.graph import END

# 여러 서브그래프를 연결하는 부모 그래프
multi_builder = StateGraph(State)
multi_builder.add_node('sub1', subgraph)   # 첫 번째 서브그래프
multi_builder.add_node('sub2', subgraph2)  # 두 번째 서브그래프
multi_builder.add_edge(START, 'sub1')
multi_builder.add_edge('sub1', 'sub2')
multi_builder.add_edge('sub2', END)

multi_graph = multi_builder.compile()

print("✅ 다중 서브그래프 부모 그래프 완료")

In [ ]:
# 그래프 구조 시각화
print("=== 다중 서브그래프 구조 ===")
print(multi_graph.get_graph().draw_mermaid())

In [ ]:
# 실행
print("=== 다중 서브그래프 실행 ===")
print(f"입력: {{'foo': 'hello'}}")
print()

result = multi_graph.invoke({'foo': 'hello'})

print()
print(f"출력: {result}")
print()
print("→ 'hello' + 'bar' + '!' = 'hellobar!'")

---

## 정리: Subgraph 직접 연결

### 핵심 개념

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Subgraph 직접 연결 방식                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   1. 서브그래프 정의 및 컴파일                                      │
│      subgraph = subgraph_builder.compile()                          │
│                                                                     │
│   2. 부모 그래프에서 노드로 추가                                    │
│      builder.add_node('name', subgraph)                             │
│                                                                     │
│   3. 공유 키를 통해 데이터 전달                                     │
│      부모 State.foo ↔ 서브그래프 State.foo                          │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 1. 공유 키가 있는 State 정의
class State(TypedDict):
    foo: str  # 공유 키

class SubgraphState(TypedDict):
    foo: str  # 공유 키
    bar: str  # 내부 키

# 2. 서브그래프 정의 및 컴파일
subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node('node', node_fn)
subgraph_builder.add_edge(START, 'node')
subgraph = subgraph_builder.compile()

# 3. 부모 그래프에서 서브그래프를 노드로 추가
builder = StateGraph(State)
builder.add_node('subgraph', subgraph)  # 직접 추가!
builder.add_edge(START, 'subgraph')
graph = builder.compile()
```

### 장단점

| 장점 | 단점 |
|------|------|
| 간단한 문법 | 상태 키 공유 필수 |
| 명확한 구조 | 유연성 낮음 |
| 자동 상태 전달 | 상태 변환 불가 |

## 다음 단계

**함수로 래핑**하여 상태 키가 달라도 서브그래프를 사용하는 방법을 배웁니다. (03번 노트북)